In [9]:
#only doing vertical spreads, not single calls,puts
#no conditionals yet

import json
import inspect
import re
from mt_parse import create_tokens,get_options
import unidecode
from IPython.display import display, Markdown, Latex

#PROCESS_ONLY_SYM = "PYPL"
PROCESS_ONLY_SYM = None



In [6]:
#with open("swing_1_9_22_20.json","r") as read_file:
#    data = json.load(read_file)
with open("cl_weekly_1_9_22_20.json","r") as read_file:
    data = json.load(read_file)
    
    

In [7]:

GREEN = '<div class="alert-success">'
YELLOW = '<div class="alert-warning">'
RED = '<div class="alert-danger">'
BLUE = '<div class="alert-info">'
END = '</div>'

    
def filter(symb,text):
    
    text_split = text.split()
    
    for i,w in enumerate(text_split):
        
        if det_symbol(i,w):
            token = Symbol(i,w)

            return token.sym == symb
            
    return False


    
def reduce_message(message):
    """ recursively concat strings of any embedded dictionaries in messages
    """

    reduced = ""

    if type(message) is list:
        for m in message:
            reduced += reduce_message(m)
        return reduced
    
    if type(message) is dict:
        if 'text'  in message.keys():
            reduced += reduce_message(message['text'])
        return reduced

    if type(message) is str:
        return message

    return reduced

def process_text(timestamp,text):
    global count
    
    
    
    if PROCESS_ONLY_SYM:
        if filter(PROCESS_ONLY_SYM,text) is False:
            return []
    
    tokens = create_tokens(timestamp,text)
    
    print("tokens:",tokens)
    

    
#    mt = YELLOW + 'tokens:' + str(tokens) + END
#    mt = '<div>' + 'tokens:' + str(tokens) + END
    #display(Markdown(mt))

#<div class="alert-warning">this is a green</div>
    #print()
    
    #print('text',text)
    #print('tokens:',tokens)

    
    
    #get_stop(timestamp,text)
    
    options = get_options(tokens,timestamp,text)
    
    
    #options = get_stop(tokens,timestamp,text,options)
    
    #print("{} options: {} {}".format(timestamp,count,options))
    
#    display(Markdown(options))

#    print(options)
    
    #count_shorted(timestamp,text); return
    
#    if options is None:
#        print('process_text none',text)

    #print()

    return options

        

    
def process_element(timestamp,element):
    
    element = reduce_message(element)
        
    element = unidecode.unidecode(element)  # convert the occasional unicode character like '–' to ascii '-'
    options = process_text(timestamp,element)
    return options
    

print('channel id:',data['id'])

options = []

for message in data['messages']:
#    if message['type'] != 'message':
#        continue

#    print(message)
    
    if 'text' in message.keys():
        options.extend(process_element(message['date'],message['text']))


#    try:
#        text = message['text']
#        process_element('date',text)
#    except:
#        print('failure:',message)
#        break

print()
print()
display(Markdown('<div class="alert-success">...</div>'))

for option in options:
    #print()
    #print("{}:option: {}".format(option['ts'],option))
    
    if "action" in option:

        if type(option['action']) is Action_shorting:
            if 'stop' in option:
                #print(option['sym'],"option has stop")
                pass
            else:
                #print(option['sym'],"option has no stop")
                pass
    else:
        if 'stop' in option:
                #print(option['stop']['sym'],"Naket Stop")
                pass

with open('actions.json','w') as outfile:
    json.dump(options,outfile)


channel id: 10015910801
tokens: [5/26:, ZM, shorting, Month[5], 2020-05-29, [180.0, 185.0], CALL, CALL, $0.45/share, Stop, P(176.22)]
tokens: [5/26:, PYPL, shorting, LotQuantity(half), Month[6], 2020-06-05, [155.0, 160.0], CALL, CALL, $0.6/share, P(1.0), Stop, P(154.56)]
tokens: [5/26:, HES, shorting, Month[5], 2020-05-29, [46.0, 44.0], PUT, PUT, P(0.21), Stop, P(46.25)]
tokens: [5/26:, AAPL, Condition, P(322.0), shorting, Month[5], 2020-05-29, [315.0, 310.0], PUT, PUT, $0.65/share, P(0.77), Stop, P(315.34)]
tokens: [5/26:, GS, shorting, Month[5], 2020-05-29, [185.0, 180.0], PUT, PUT, P(0.5), Stop, P(185.21)]
tokens: [Adj, 5/26:, ZM, shorted, Month[5], 2020-05-29, [180.0, 185.0], CALL, CALL, $0.45/share, 5/26:, MidPoint, Close, $0.25/share, Stop, P(168.22)]
tokens: [5/26:, W, shorting, Month[5], 2020-05-29, [170.0, 175.0], CALL, CALL, $0.6/share, Stop, P(168.72)]
tokens: [Adj, 5/26:, ZM, shorted, Month[5], 2020-05-29, [180.0, 185.0], CALL, CALL, $0.45/share, 5/27:, Cover, Condition, Pe

<div class="alert-success">...</div>

NameError: name 'Action_shorting' is not defined

<div class="alert-warning">this is a green</div>

<div class="alert-danger">this is a green</div>

foo <font color='red'>bar</font> foo

In [14]:
i = False
i = 2
if (i = 2):
    print("exis")

SyntaxError: invalid syntax (<ipython-input-14-a612a3f8264a>, line 3)

In [ ]:
print(options[5])

In [ ]:
if 'action' in options[5]:
    print('abc')

In [ ]:
{'timestamp': '2020-05-26T06:52:24', 'action': 'shorting', 'sym': 'ZM', 'spread_type': 'CALL', 'strike_date': '2020-05-29', 'strikes': [180.0, 185.0], 'stop': {'timestamp': '2020-05-26T06:52:24', 'sym': 'ZM', 'type': 'price', 'price': 176.22}}